In [1]:
#!/usr/bin/env python
import pickle
import pandas as pd
from pymongo import MongoClient


In [2]:

# --- Connect to MongoDB ---
client = MongoClient("mongodb://localhost:27017/")
db = client["mydatabase"]


In [4]:
help(db)

Help on Database in module pymongo.synchronous.database object:

class Database(pymongo.common.BaseObject, typing.Generic)
 |  Database(
 |      client: 'MongoClient[_DocumentType]',
 |      name: 'str',
 |      codec_options: 'Optional[CodecOptions[_DocumentTypeArg]]' = None,
 |      read_preference: 'Optional[_ServerMode]' = None,
 |      write_concern: 'Optional[WriteConcern]' = None,
 |      read_concern: 'Optional[ReadConcern]' = None
 |  ) -> 'None'
 |
 |  Method resolution order:
 |      Database
 |      pymongo.common.BaseObject
 |      typing.Generic
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __bool__(self) -> 'NoReturn'
 |
 |  __eq__(self, other: 'Any') -> 'bool'
 |      Return self==value.
 |
 |  __getattr__(self, name: 'str') -> 'Collection[_DocumentType]'
 |      Get a collection of this database by name.
 |
 |      Raises InvalidName if an invalid collection name is used.
 |
 |      :param name: the name of the collection to get
 |
 |  __getitem__(self, n

In [5]:

# Optional: Drop existing collections to avoid duplicates
db.refined_profiles.drop()
db.refined_cluster.drop()
db.vectorized_refined.drop()
db.categories.drop()
db.probability.drop()


In [7]:

# --- Load Pickle Files ---
with open("refined_profiles.pkl", "rb") as fp:
    refined_profiles = pickle.load(fp)

with open("refined_cluster.pkl", "rb") as fp:
    refined_cluster = pickle.load(fp)

with open("vectorized_refined.pkl", "rb") as fp:
    vectorized_refined = pickle.load(fp)

with open("categories.pkl", "rb") as fp:
    categories = pickle.load(fp)

with open("probability.pkl", "rb") as fp:
    probability = pickle.load(fp)


In [8]:

# --- Prepare Data for Insertion ---
def dataframe_to_records(data):
    if isinstance(data, pd.DataFrame):
        return data.to_dict(orient="records")
    return data


In [9]:

refined_profiles_records = dataframe_to_records(refined_profiles)
refined_cluster_records = dataframe_to_records(refined_cluster)
vectorized_refined_records = dataframe_to_records(vectorized_refined)


C:\Users\Anush\AppData\Local\Temp\ipykernel_6200\1158291107.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return data.to_dict(orient="records")


In [10]:

# --- Insert into MongoDB ---
if isinstance(refined_profiles_records, list):
    db.refined_profiles.insert_many(refined_profiles_records)
else:
    db.refined_profiles.insert_one(refined_profiles_records)

if isinstance(refined_cluster_records, list):
    db.refined_cluster.insert_many(refined_cluster_records)
else:
    db.refined_cluster.insert_one(refined_cluster_records)

if isinstance(vectorized_refined_records, list):
    db.vectorized_refined.insert_many(vectorized_refined_records)
else:
    db.vectorized_refined.insert_one(vectorized_refined_records)

# For categories and probability, we assume they are dictionaries.
db.categories.insert_one(categories)
db.probability.insert_one(probability)

print("Data inserted to MongoDB successfully.")


InvalidDocument: Invalid document {'Movies': ['Adventure', 'Action', 'Drama', 'Comedy', 'Thriller', 'Horror', 'RomCom', 'Musical', 'Documentary'], 'Religion': ['Catholic', 'Christian', 'Jewish', 'Muslim', 'Hindu', 'Buddhist', 'Spiritual', 'Other', 'Agnostic', 'Atheist'], 'Music': ['Rock', 'HipHop', 'Pop', 'Country', 'Latin', 'EDM', 'Gospel', 'Jazz', 'Classical'], 'Politics': ['Liberal', 'Progressive', 'Centrist', 'Moderate', 'Conservative'], 'Social Media': ['Facebook', 'Youtube', 'Twitter', 'Reddit', 'Instagram', 'Pinterest', 'LinkedIn', 'SnapChat', 'TikTok'], 'Sports': ['Football', 'Baseball', 'Basketball', 'Hockey', 'Soccer', 'Other'], 'Age': array([20, 31, 24, ..., 23, 20, 38], shape=(6600,)), '_id': ObjectId('67af2b2a087f5801f41d9d2f')} | cannot encode object: array([20, 31, 24, ..., 23, 20, 38], shape=(6600,)), of type: <class 'numpy.ndarray'>

In [13]:
#!/usr/bin/env python
import pickle
import pandas as pd
from pymongo import MongoClient

# --- Connect to MongoDB ---
client = MongoClient("mongodb://localhost:27017/")
db = client["mydatabase"]

# Optional: Clear existing collections (to avoid duplicates)
db.refined_profiles.drop()
db.refined_cluster.drop()
db.vectorized_refined.drop()
db.categories.drop()
db.probability.drop()

# --- Load Pickle Files ---
with open("refined_profiles.pkl", "rb") as fp:
    refined_profiles = pickle.load(fp)

with open("refined_cluster.pkl", "rb") as fp:
    refined_cluster = pickle.load(fp)

with open("vectorized_refined.pkl", "rb") as fp:
    vectorized_refined = pickle.load(fp)

with open("categories.pkl", "rb") as fp:
    categories = pickle.load(fp)

with open("probability.pkl", "rb") as fp:
    probability = pickle.load(fp)

# --- Convert DataFrames to Records (if applicable) ---
def dataframe_to_records(data):
    if isinstance(data, pd.DataFrame):
        return data.to_dict(orient="records")
    return data

refined_profiles_records = dataframe_to_records(refined_profiles)
refined_cluster_records = dataframe_to_records(refined_cluster)
vectorized_refined_records = dataframe_to_records(vectorized_refined)

# --- Insert into MongoDB ---
if isinstance(refined_profiles_records, list):
    db.refined_profiles.insert_many(refined_profiles_records)
else:
    db.refined_profiles.insert_one(refined_profiles_records)

if isinstance(refined_cluster_records, list):
    db.refined_cluster.insert_many(refined_cluster_records)
else:
    db.refined_cluster.insert_one(refined_cluster_records)

if isinstance(vectorized_refined_records, list):
    db.vectorized_refined.insert_many(vectorized_refined_records)
else:
    db.vectorized_refined.insert_one(vectorized_refined_records)

print("Data inserted to MongoDB successfully.")


C:\Users\Anush\AppData\Local\Temp\ipykernel_6200\893838558.py:36: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return data.to_dict(orient="records")


Data inserted to MongoDB successfully.
